# Les 5 aliments les plus caloriques en kcal/kg

In [1]:
import pandas as pd

In [2]:
#Récupération des données utiles
data_population = pd.read_csv('fr_population.csv')
data_animaux = pd.read_csv('fr_animaux.csv')
data_vegetaux = pd.read_csv('fr_vegetaux.csv')

#On gère nos noms de colonnes via des variables pour faciliter la maintenance du code
code_zone = "Code zone"
zone = 'Zone'
valeur = 'Valeur'
element = "Élément"
code_domaine = 'Code Domaine'
domaine = 'Domaine'
code_element = 'Code Élément'
code_produit = 'Code Produit'
produit = 'Produit'
code_annee = 'Code année'
annee = 'Année'
symbole = 'Symbole'
description_symbole = 'Description du Symbole'
unite = 'Unité'
energie_poids = 'Ratio énergie/poids en Kcal/Kg'

data_population = data_population[data_population[code_zone] != 351]
data_animaux = data_animaux[data_animaux[code_zone] != 351]
data_vegetaux = data_vegetaux[data_vegetaux[code_zone] != 351]
data_animaux.shape

(36908, 14)

In [3]:
data_animaux = data_animaux.drop([code_zone, code_domaine, domaine, code_element, code_produit, code_annee, annee, symbole, description_symbole ], axis = 1)
data_vegetaux = data_vegetaux.drop([code_zone, code_domaine, domaine, code_element, code_produit, code_annee, annee, symbole, description_symbole ], axis = 1)
data_vegetaux.shape

(104011, 5)

In [4]:
data_prod = data_animaux.append(data_vegetaux).reset_index()
data_prod.head()

,index,Zone,Élément,Produit,Unité,Valeur
0,0,Afghanistan,Production,Viande de Bovins,Milliers de tonnes,134.00
1,1,Afghanistan,Importations - Quantité,Viande de Bovins,Milliers de tonnes,6.00
2,2,Afghanistan,Disponibilité intérieure,Viande de Bovins,Milliers de tonnes,140.00
3,3,Afghanistan,Nourriture,Viande de Bovins,Milliers de tonnes,140.00
4,4,Afghanistan,Disponibilité alimentaire en quantité (kg/pers...,Viande de Bovins,kg,4.59


In [5]:
#Fonction qui récupère la population d'un pays donné
def recup_pop(pays) :
    pop = data_population[data_population[zone] == pays][valeur].values
    pop = int(pop * 1000)
    return pop

In [6]:
#Fonction qui permet de calculer le ratio energie/poids d'un produit dans un pays
def calculer_energie_poids(data) :
    data_ratio_values = []
    mask1 = data[element] == 'Nourriture'
    mask3 = data[element] == 'Disponibilité alimentaire (Kcal/personne/jour)'
    data = data[mask1 | mask3]
    kg = 0
    kcal = 0
    for z, p, u, v in zip(data[zone], data[produit], data[unite], data[valeur]) :
        div = False
        if u == 'Milliers de tonnes' :
            kg = v * 1000000
        else :
            kcal = v
            div = True
        if div == True :
            if kg == 0 :
                ratio = 0
            else :
                pop = recup_pop(z)
                kcal *= pop * 365
                ratio = round((kcal / kg), 2)
                ratio_values =  [z, p, ratio]
                data_ratio_values.append(ratio_values)
    data_ratio = pd.DataFrame(data_ratio_values, columns = [zone, produit, energie_poids])
    data_ratio = data_ratio[data_ratio[energie_poids] != 0]
    return data_ratio

In [7]:
data_ratio_kcal = calculer_energie_poids(data_prod)
data_ratio_kcal.head()

,Zone,Produit,Ratio énergie/poids en Kcal/Kg
0,Afghanistan,Viande de Bovins,2150.64
1,Afghanistan,Viande d'Ovins/Caprins,2304.64
2,Afghanistan,Viande de Volailles,1338.18
3,Afghanistan,"Viande, Autre",929.29
4,Afghanistan,Abats Comestible,1052.03


In [8]:
#On fait la moyenne du ratio energie/kg pour chaque produit et on les classe de façon décroissante
data_ratio_kcal = data_ratio_kcal.groupby(produit).mean(energie_poids).sort_values(energie_poids, ascending = False)

In [9]:
#Réponse à la question 5
data_ratio_kcal.head(5)

,Ratio énergie/poids en Kcal/Kg
Produit,
Huiles de Foie de Poisso,9982.930000
Huile de Sésame,9433.745429
Huile de Son de Riz,9361.549167
Huiles de Poissons,9003.716667
Huile d'Arachide,8935.765921


In [10]:
data_ratio_kcal.to_csv("P3_05_données.csv")